## Notebook para la exploración de los ficheros de datos del TFM
### Script para la exploración de los datasets para el TFM: Optimización de la toma de decisiones empresarial
### mediante el análisis de datos

### 1. Carga de los ficheros y exploración de las variables

In [51]:
# Importación de librerías
import pandas as pd

In [52]:
# Lectura de los .CSV
df = pd.read_csv('C:/Users/UX530/Desktop/TFM-GIT/Origin_Files/extended_cto_2025-09-30_09_35.csv', delimiter = ';')

### Se hará una copia del dataframen original

In [53]:
df_CTO = df.copy()  # Se hará una copia del dataframen original

In [54]:
# Cabecera del dataset
df_CTO.head()

,ID,CTO,Código CTO,Estado,Tipo de ampliación,Ticket Jira,EC,Fecha de solicitud,Fecha de ejecución,Fecha de documentación,...,Nueva CTO,Rechazos (iteraciones),Provincia,Población,Zona,Fase,Cluster,Proveedor OLT,Observaciones,Activo
0,1,489-46-008078.2,48946-008078.02,FINALIZADA,PREVENTIVA-1,NaN,ELECNOR,2019-01-18,2019-03-28 09:26:11,2019-03-28,...,NaN,NaN,VALENCIA,ALBALAT DE LA RIBERA,3.0,NaN,489-46-008-004-CL,Masmovil,NaN,Sí
1,2,489-46-011192,48946-011192,CANCELADA,F07,NaN,ELECNOR,2019-01-18,NaN,NaN,...,489-46-011094,NaN,CASTELLON,ALMENARA,3.0,NaN,489-46-011-017-CL,Masmovil,BALANCEO DE HH,Sí
2,3,489-46-011225,48946-011225,FINALIZADA,F07,NaN,ELECNOR,2019-02-01,2019-02-15 12:50:03,2019-03-05,...,NaN,NaN,VALENCIA,ALBERIC,3.0,NaN,489-46-011-004-CL,Masmovil,"CARGADA EN BBDD, ELIMINAR HH",Sí
3,5,489-46-011118,48946-011118,CANCELADA,PREVENTIVA-1,NaN,ELECNOR,2019-01-18,NaN,NaN,...,NaN,NaN,VALENCIA,ALBERIC,3.0,NaN,489-46-011-038-CL,Masmovil,NaN,Sí
4,6,489-46-011158.1,48946-011158.01,FINALIZADA,PREVENTIVA-1,NaN,ELECNOR,2019-01-18,2019-02-15 13:09:21,2019-03-05,...,NaN,NaN,VALENCIA,ALBERIC,3.0,NaN,489-46-011-022-CL,Masmovil,CARGADA EN BBDD,Sí


In [55]:
# Número de filas y columnas
print("Número de filas y columnas:", df_CTO.shape)

# Nombre de las columnas
print("Nombres de las columnas:", df_CTO.columns.tolist())

# Tipo de datos de cada columna
print("Tipos de datos:\n", df_CTO.dtypes)

# Información general del DataFrame
print("Información del DataFrame:" , df_CTO.info())

Número de filas y columnas: (11505, 28)
Nombres de las columnas: ['ID', 'CTO', 'Código CTO', 'Estado', 'Tipo de ampliación', 'Ticket Jira', 'EC', 'Fecha de solicitud', 'Fecha de ejecución', 'Fecha de documentación', 'Fecha de parada', 'Fecha de reanudación', 'Fecha Documentación Rechazada', 'Fecha  Documentación Reparada', 'Fecha de finalización', 'Fecha Cancelación', 'SLA', 'Geotipo', 'Nueva CTO', 'Rechazos (iteraciones)', 'Provincia', 'Población', 'Zona', 'Fase', 'Cluster', 'Proveedor OLT', 'Observaciones', 'Activo']
Tipos de datos:
 ID                                 int64
CTO                               object
Código CTO                        object
Estado                            object
Tipo de ampliación                object
Ticket Jira                       object
EC                                object
Fecha de solicitud                object
Fecha de ejecución                object
Fecha de documentación            object
Fecha de parada                   object
Fecha d

In [56]:
# Nos da los valores de "CTO" que se repiten más de una vez y cuántas veces
df_CTO['CTO'].value_counts()[df_CTO['CTO'].value_counts() > 1]

CTO
489-28-091111.1    7
489-22-112392.2    5
489-45-183118      5
489-28-145M00      5
489-13-028404.1    4
                  ..
489-20-K07482      2
489-48-K12003      2
489-50-235420.1    2
489-13-028354.1    2
489-23-094109      2
Name: count, Length: 614, dtype: int64

In [57]:
# Calacula el porcentaje de valores nulos en cada columna, y nos quedamos con las que tienen más del 60%
# serán columnas candidatas a eliminar del modelo. 
Columns_delete = (df_CTO.isnull().mean() * 100)[df_CTO.isnull().mean() * 100 > 60].index.tolist()
Columns_delete

['Ticket Jira',
 'Fecha de parada',
 'Fecha de reanudación',
 'Fecha Documentación Rechazada',
 'Fecha  Documentación Reparada',
 'Fecha Cancelación',
 'Nueva CTO',
 'Rechazos (iteraciones)',
 'Observaciones']